In [328]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import certifi
import urllib3
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from urllib.parse import urljoin
import random
from typing import List
import re

notas 
revisar de la lista de urls las paginas que no tienen paginacion y ver cuales se puede y cuales no 
*  caso easy no se encuentra paginacion
*  revisar url hilti, no las reconoce
*  revisar icsa e incoresa no reconoce que tenga paginacion
*  revisar ferrepat no reconoce ningun link


In [329]:
ruta=r'/home/sebastian/Documentos/programas/Webscrapping/all_links.xlsx'
df_urls=pd.read_excel(ruta,header=0,sheet_name='urls')
df_urls['url final']=df_urls['url final'].str.replace(" ", "", regex=False)
df_urls['url final'] = df_urls['url final'].str.strip()
df_urls.head()

,Code Country,Country,Name,Information,Type Pagination,Note,Secuencia de paginacion,url final
0,ARG,Argentina,abc sa,SBD & OTHER,scrolldown,mi,no pag,https://www.abc-sa.com.ar/catalogue/all
1,ARG,Argentina,Black & Decker,SBD,page,sec+1,1,https://ar.blackanddecker.global/productos/her...
2,ARG,Argentina,Black & Decker,SBD,page,sec+1,1,https://ar.blackanddecker.global/productos/her...
3,ARG,Argentina,cetrogar,SBD & OTHER,page,sec+1,1,https://www.cetrogar.com.ar/herramientas.html?...
4,ARG,Argentina,Dewalt,SBD,page,sec+1,1,https://ar.dewalt.global/productos/accesorios?...


In [330]:
# Verificacion de si es producto
posibles_subcadenas = [
    # Generales en e-commerce
    '/product/', '/products/', '/producto/', '/productos/',
    '/item/', '/items/', '/detalle/', '/detail/',
    '/sku/', '/articulo/', '/artículos/',

    # Términos comunes en español e inglés
    '/detalle-producto/', '/ver-producto/', '/ver_producto/',
    '/viewproduct/', '/product-detail/', '/product_info/',

    # Prefijos o patrones típicos
    '-p-', '-prod-', '-item-', '-sku-', '-detalle-',

    # Patrones Amazon
    '/dp/', '/gp/product/',

    # Patrones MercadoLibre (por país)
    '/mla-', '/mlm-', '/mlc-', '/mlv-', '/mlu-', '/mlb-', '/mls-',  # Argentina, México, Chile, Venezuela, Uruguay, Brasil, Colombia

    # Easy y Sodimac (comunes en LATAM)
    '/producto/', '/productos/', '/sku/', '/ficha/', '/ficha-producto/',

    # Stanley, DeWalt, Bosch, Makita, etc.
    '/tools/', '/catalog/', '/categories/', '/details/', '/item-details/',

    # Otros posibles patrones semánticos
    '/shop/', '/buy/', '/comprar/', '/oferta/', '/ofertas/', '/promo/', '/promocion/',
    
    # Extensiones finales sospechosas
    '.html', '.htm'
  ]

# validar_url
session = requests.Session()
user_agents = [
    # Lista simple para rotar headers
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64)...',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)...',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64)...',
    # Puedes expandirla con más UA reales
]


In [331]:
def contiene_paginacion(soup):
    '''Verifica si alguna etiqueta <a> o button contiene indicios de paginacion
    arg :soup
    return valor(0,1,2) respuesta
    '''
    # Lista extendida y homogénea (en minúsculas)
    palabras_paginacion = [
        'siguiente', 'sig', '›', '»', '→', 'adelante', 'próximo',
        'next', 'forward', '>', '>>', 'seguinte'
    ]

    # Buscar todos los enlaces y botones con texto que podría ser paginación
    elementos = soup.find_all(['a', 'button', 'li'])

    for el in elementos:
        texto = el.get_text(strip=True).lower()
        if any(p in texto for p in palabras_paginacion):
            print(f'contiene paginacion')
            return True
        
    return False

def validar_url(url, session=session, timeout=20):
    '''Valida cada url paginada para saber si se puede o no ingresar a esta, y ademas si esta es o no la ultima pagina, esto mediante la verificacion si tiene o no un boton que permita cambiar
     de pag
      arg: 
        * url: url paginada
         *session:mantiene la sesion activa
          *timeout: tiempo de espera
     '''
    try:
        headers = {
            'User-Agent': random.choice(user_agents),
            'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8'
        }
        respuesta = session.get(url, headers=headers, verify=False, timeout=timeout, allow_redirects=True)

        if respuesta.status_code == 200:
            soup = BeautifulSoup(respuesta.text, 'html.parser')
            
            if contiene_paginacion(soup):
                return 0, respuesta
            else:
                return 1, respuesta
        else:
            return 2, None
    except Exception as e:
        return 3, None


In [332]:

# Precompilamos patrones
PATRONES_PRODUCTO = re.compile(r'(product|item|skcard|detail)', re.I)
PATRONES_FILTROS = re.compile(r'(category|categories|filter|#)', re.I)
ATRIBUTOS_PRODUCTO = ['data-product-id', 'data-sku', 'data-id', 'data-item-id']

def es_link_de_producto(a_tag, posibles_subcadenas):
    href = a_tag['href'].lower()
    clases = a_tag.get('class', [])
    atributos = a_tag.attrs

    # Condición: contiene palabra clave
    contiene_subcadena = any(sub in href for sub in posibles_subcadenas)

    # Condición: clase relacionada a producto
    clase_relevante = any(PATRONES_PRODUCTO.search(clase) for clase in clases)

    # Condición: atributos HTML de producto
    tiene_atributo = any(attr in atributos for attr in ATRIBUTOS_PRODUCTO)

    # Condición: no es un filtro ni ancla
    no_es_filtro = not PATRONES_FILTROS.search(href)

    return (contiene_subcadena or clase_relevante or tiene_atributo) and no_es_filtro

def obtener_links_web_paginada(respuesta, url, posibles_subcadenas):
    soup = BeautifulSoup(respuesta.text, 'html.parser')
    links = set()

    for a in soup.find_all('a', href=True):
        if es_link_de_producto(a, posibles_subcadenas):
            full_url = urljoin(url, a['href'])
            links.add(full_url)

    return list(links)


In [333]:
def extraer_links_url(url_base: str, sec_pag: int, posibles_subcadenas: List[str], pausa: int = 10, max_errores: int = 2) -> List[str]:
    """
    Extrae enlaces de productos de todas las páginas asociadas a una URL base paginada.

    Parámetros:
    - url_base: string con 'num_pag' como marcador para el número de página.
    - sec_pag: incremento por página (1, 10, etc.).
    - posibles_subcadenas: lista de palabras clave que indican que un link es de producto.
    - pausa: segundos de espera entre solicitudes (default: 10).
    - max_errores: número máximo de fallos consecutivos antes de detener (default: 2).

    Retorna:
    - Lista de URLs extraídas.
    """
    
    errores_consecutivos = 0
    numero_pagina = 0
    links_extraidos = []

    while errores_consecutivos < max_errores:
        # Reemplazo del marcador en la URL
        url = url_base.replace('num_pag', str(numero_pagina)) if 'num_pag' in url_base else url_base

        estado, respuesta = validar_url(url)
        print(f'estado de la url {url} es :{estado}')
        if estado <=1:  # Estado 0 -1significa respuesta válida
            links_pagina = obtener_links_web_paginada(respuesta, url, posibles_subcadenas)
            print(f"[✅] Página {numero_pagina}: {len(links_pagina)} links encontrados en {url}")

            if not links_pagina:
                print(f"[🛑] Página {numero_pagina} sin contenido. Deteniendo scraping.")
                break

            if estado == 1:
                errores_consecutivos +=1
                print(f'la url :{url} en su pagina {sec_pag} no tiene informacion, numero de errores {errores_consecutivos}')
            else:
                errores_consecutivos +=0   # Resetear contador de errores
                numero_pagina += sec_pag
    
            url_new=url_base.replace('num_pag', str(numero_pagina)) if 'num_pag' in url_base else url_base

           
            if url==url_new:
                errores_consecutivos+=1
                print(f'la url :{url} ya se evaluo, numero de errores {errores_consecutivos}')
            else:
                print(f'la respuesta de :{url}  cambio')
            
            links_extraidos.extend(links_pagina)
    
    
        else:
            errores_consecutivos += 1
            print(f' estado para {url} es :{estado}')
            print(f"[⚠️] Error en {url}. Intento {errores_consecutivos} de {max_errores}")

        time.sleep(pausa)

    return list(set(links_extraidos))

In [334]:
df_urls.columns

Index(['Code Country', 'Country', 'Name', 'Information', 'Type Pagination',
       'Note', 'Secuencia de paginacion', 'url final'],
      dtype='object')

In [335]:

def construir_dataframe_links(df_urls: pd.DataFrame, url_base: str, lista_links: List[str]) -> pd.DataFrame:
    """
    Construye un DataFrame con metadatos asociados a una URL base y una lista de links de productos.

    Args:
        df_urls (pd.DataFrame): DataFrame que contiene las URLs base y sus metadatos.
        url_base (str): URL base usada para extraer productos.
        lista_links (List[str]): Lista de URLs de productos extraídos.

    Returns:
        pd.DataFrame: DataFrame con cada link asociado a su contexto (país, nombre, paginación, etc).
    """
    # Validar que la URL base esté presente
    coincidencias = df_urls[df_urls['url final'] == url_base]
    
    if coincidencias.empty:
        print(f"[⚠️] construir_dataframe_links: No se encontró metadata para URL base: {url_base}")
        return pd.DataFrame()  # Devuelve DataFrame vacío si no hay coincidencias

    fila = coincidencias.iloc[0]

    # Crear el DataFrame final replicando los metadatos por cada link
    df_resultado = pd.DataFrame({
        'url final': lista_links,
        'Code Country': fila['Code Country'],
        'Country': fila['Country'],
        'Name': fila['Name'],
        'Information': fila['Information'],
        'Type Pagination': fila['Type Pagination'],
        'Note': fila['Note'],
        'Secuencia de paginacion': fila['Secuencia de paginacion']
    })

    print(f"construir_dataframe_links: Se creó el DataFrame con {len(lista_links)} links para {fila['Country']} - {fila['Name']}")
    return df_resultado


In [336]:
def procesar_fila(df_urls: pd.DataFrame, fila: pd.Series) -> pd.DataFrame:
    url_base = fila['url final']
    try:
        sec_pag = int(fila['Secuencia de paginacion'])
    except ValueError:
        print(f"[⚠️] Error convirtiendo secuencia de paginación a int: {fila['Secuencia de paginacion']}")
        sec_pag = 1  # Valor por defecto

    links = extraer_links_url(
        url_base=url_base,
        sec_pag=sec_pag,
        posibles_subcadenas=posibles_subcadenas,
        pausa=5  # Puedes ajustar esto
    )

    if links:
        return construir_dataframe_links(df_urls, url_base, links)
    else:
        return pd.DataFrame()  # Devuelve vacío si no se extrajo nada
    
    
def extraer_links_todas_las_urls(df_urls: pd.DataFrame, max_workers: int = 50) -> pd.DataFrame:
    resultados = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futuros = [
            executor.submit(procesar_fila, df_urls, fila)
            for _, fila in df_urls.iterrows()
        ]

        for futuro in tqdm(as_completed(futuros), total=len(futuros), desc="Procesando URLs"):
            resultado = futuro.result()
            if not resultado.empty:
                resultados.append(resultado)

    if resultados:
        df_consolidado = pd.concat(resultados, ignore_index=True)
        print(f"[✅] Se extrajeron {len(df_consolidado)} links en total.")
        return df_consolidado
    else:
        print("[⚠️] No se extrajo ningún link.")
        return pd.DataFrame()


In [337]:
df_prueba=df_urls[df_urls['url final']=='https://incoresa.com.pe/tienda/']
#df_prueba=df_urls[2:4]
df_prueba

,Code Country,Country,Name,Information,Type Pagination,Note,Secuencia de paginacion,url final
1404,PER,Peru,Incoresa,SBD & OTHER,tiene paginaciion pero no cambia la url por pag,no pag,no pag,https://incoresa.com.pe/tienda/


In [338]:
df_links_webpaginadas=extraer_links_todas_las_urls(df_prueba, max_workers=4)

[⚠️] Error convirtiendo secuencia de paginación a int: no pag


Procesando URLs:   0%|          | 0/1 [00:00<?, ?it/s]

estado de la url https://incoresa.com.pe/tienda/ es :1
[✅] Página 0: 31 links encontrados en https://incoresa.com.pe/tienda/
la url :https://incoresa.com.pe/tienda/ en su pagina 1 no tiene informacion, numero de errores 1
la url :https://incoresa.com.pe/tienda/ ya se evaluo, numero de errores 2


Procesando URLs: 100%|██████████| 1/1 [00:05<00:00,  5.90s/it]

construir_dataframe_links: Se creó el DataFrame con 31 links para Peru - Incoresa
[✅] Se extrajeron 31 links en total.


In [316]:
x='https://www.troybilt.com/en_US/three-stage-snow-blowers/vortex-2610-snow-blower/31AH5DP7B66.html?fitsOnModel=false'
df_links_webpaginadas[df_links_webpaginadas['url final']==x]

,url final,Code Country,Country,Name,Information,Type Pagination,Note,Secuencia de paginacion
6,https://www.troybilt.com/en_US/three-stage-sno...,GLOBAL,Global,Troy-Bilt,SBD,load more,no pag,no pag
